In [1]:
import pandas as pd
import numpy as np
import ast
import math
from pyvis.network import Network
import sys
sys.path.append('../')
from modules import utils, constants

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
# test_df = pd.read_csv('../test_dfs/dueling_ddqn_per_biopsy_3_best_acc_median_63_100000000.csv')
test_df = pd.read_csv('../test_dfs/dueling_ddqn_per_biopsy_3_best_wpahm_median_9_1_42_100000000.csv')
test_df = test_df[test_df.y_pred==1]
test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,14.0,0.884315,1.0,1.0,"['ana', 'proteinuria', 'joint_involvement', 'p...",0.0,1.0
2,2.0,5.0,0.963899,1.0,1.0,"['ana', 'proteinuria', 'pericardial_effusion',...",0.0,1.0
4,4.0,4.0,0.973561,1.0,1.0,"['ana', 'proteinuria', 'joint_involvement', 'L...",0.0,1.0
7,7.0,10.0,0.924583,1.0,1.0,"['ana', 'proteinuria', 'pericardial_effusion',...",0.0,1.0
9,9.0,5.0,0.963899,1.0,1.0,"['ana', 'proteinuria', 'pericardial_effusion',...",0.0,1.0


In [3]:
utils.test(test_df.y_actual, test_df.y_pred)[0]

98.04711717296962

In [4]:
testing_df = pd.read_csv('../new_data/test_set_constant.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,4,1,0,0,0,1,0,0,1


In [5]:
training_df = pd.read_csv('../new_data/train_set_basic.csv')
training_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
1,1,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,1,0,...,1,3,0,0,0,0,0,0,0,1


#### Useful functions

In [6]:
def generate_tuple_dict(df):
    frequency_dict = {}
    for traj in df.trajectory:
        if traj in frequency_dict.keys():
            frequency_dict[traj] += 1
        else:
            frequency_dict[traj] = 1
    overall_tup_dict = {}
    for key, value in frequency_dict.items():
        new_key = ast.literal_eval(key)
        for tup in zip(new_key, new_key[1:]):
            if tup in overall_tup_dict.keys():
                overall_tup_dict[tup] += value
            else:
                overall_tup_dict[tup] = value
    return overall_tup_dict

In [7]:
def create_sankey_df(df):
    overall_tup_dict = generate_tuple_dict(df)
    sankey_df = pd.DataFrame()
    sankey_df['source'] = [i[0] for i in overall_tup_dict.keys()]
    sankey_df['target'] = [i[1] for i in overall_tup_dict.keys()]
    sankey_df['value'] = list(overall_tup_dict.values())
    sankey_df['link_type'] = sankey_df['target'].apply(lambda i: 'terminal' if i in ['No lupus', 'Lupus', 'Inconclusive diagnosis'] else 'non_terminal')
    return sankey_df

In [8]:
def draw_pyvis_network(pathways_df, pathway_type='network'): #second type is trajectory 
    pathways_df['type'] = 'directed'
    start_node = pathways_df.iloc[0]['source']
    all_nodes = list(set(pathways_df.source.unique().tolist() + pathways_df.target.unique().tolist()))
    terminal_nodes = list(set(pathways_df[pathways_df.link_type=='terminal'].target))
    non_terminal_nodes = [i for i in all_nodes if i not in terminal_nodes and i!=start_node]
    
    #change get_net to something else since it satnds for Game of Thrones
    got_net = Network(notebook=True, height='600px', width='100%', directed=True, cdn_resources='in_line')
    got_net.add_node(start_node, color='purple', size=20)
    got_net.add_nodes(non_terminal_nodes, size=[15]*len(non_terminal_nodes), color=['blue']*len(non_terminal_nodes))
    got_net.add_nodes(terminal_nodes, color=['green']*len(terminal_nodes), size=[20]*len(terminal_nodes))
    for src, target, value, edge_thresh in zip(pathways_df.source, pathways_df.target, pathways_df.value, pathways_df.edge_threshold):
        if pathway_type =='network':
            if value > threshold:
                if math.isnan(edge_thresh):
                    got_net.add_edge(src, target, value=value, color='red')
                else:
                    got_net.add_edge(src, target, value=value, color='red', label=str(int(edge_thresh)))
            else:
                got_net.add_edge(src, target, value=value, color='blue')
        elif pathway_type =='trajectory': #It's just one trajectory
            if isinstance(edge_thresh, str):
                got_net.add_edge(src, target, value=value, color='blue', label=edge_thresh)
            else:
                got_net.add_edge(src, target, value=value, color='blue', label=str(int(edge_thresh)))
        else:
            print('Unknown pathway type')
    return got_net

In [9]:
def get_threshold_value(df, source): #source e.g. 'ana'
    threshold_values = df[source].unique().tolist()
    if len(threshold_values) == 1:
        return int(threshold_values[0])
    else:
        return str(threshold_values)

In [10]:
def draw_one_trajectory(pred_df, trajectory): #pred_df = pred_no_lupus par exemple
    trajectory_df = pred_df[pred_df.trajectory == trajectory]
    trajectory_testing_df = testing_df.loc[trajectory_df.index]
    trajectory_pathways_df = create_sankey_df(trajectory_df)
    trajectory_pathways_df['edge_threshold'] = [get_threshold_value(trajectory_testing_df, source) for source in trajectory_pathways_df.source]
    trajectory_net = draw_pyvis_network(trajectory_pathways_df, 'trajectory')   
    return trajectory_net

#### Commonest pathway 1

In [11]:
commonest_path = test_df.trajectory.value_counts().index[0]
commonest_path_net = draw_one_trajectory(test_df, commonest_path)
commonest_path_net.show('Example.html')

Example.html


In [14]:
len(test_df[test_df.trajectory == commonest_path])

1119

In [15]:
utils.get_pathway_score(commonest_path)

0.8659611992945326

In [16]:
# commonest_path_training_df = training_df[training_df.ana==0]
commonest_path_training_df = training_df[((((training_df.ana==1) & (training_df.pericardial_effusion==0)) & (training_df.anti_smith_antibody==0)) & (training_df.proteinuria==1))& (training_df.joint_involvement==1)]
len(commonest_path_training_df)

2680

In [17]:
len(commonest_path_training_df[commonest_path_training_df.label==1])/len(commonest_path_training_df)*100

100.0

#### Commonest pathway 2

In [ ]:
commonest_path2 = test_df.trajectory.value_counts().index[1]
commonest_path2_net = draw_one_trajectory(test_df, commonest_path2)
commonest_path2_net.show('Example.html')

In [ ]:
len(test_df[test_df.trajectory == commonest_path2])

In [ ]:
utils.get_pathway_score(commonest_path2)

In [ ]:
# commonest_path2_training_df = training_df[(((((training_df.ana==1) & (training_df.proteinuria==0)) & (training_df.pericardial_effusion==0)) & (training_df.joint_involvement==0)) & (training_df.pleural_effusion==0)) & (training_df.anti_dsdna_antibody==0)]
len(commonest_path2_training_df)

In [ ]:
len(commonest_path2_training_df[commonest_path2_training_df.label==0])/len(commonest_path2_training_df)*100

#### Commonest pathway 3

In [18]:
commonest_path3 = test_df.trajectory.value_counts().index[2]
commonest_path3_net = draw_one_trajectory(test_df, commonest_path3)
commonest_path3_net.show('Example.html')

Example.html


In [19]:
len(test_df[test_df.trajectory == commonest_path3])

482

In [20]:
utils.get_pathway_score(commonest_path3)

0.7671957671957672

In [21]:
# commonest_path3_training_df = training_df[(((((((((((((training_df.ana==1) & (training_df.proteinuria==1)) & (training_df.joint_involvement==0)) & (training_df.pericardial_effusion==0)) & (training_df.anti_dsdna_antibody==0)) & (training_df.delirium==0)) &(training_df.pleural_effusion==0)) & (training_df.fever==0)) &(training_df.psychosis==0)) & (training_df.anti_smith_antibody==0)) & (training_df.low_c3==0)) & (training_df.oral_ulcers==0)) & (training_df.lupus_anti_coagulant==0)) & (training_df.low_c4==0)]
commonest_path3_training_df = training_df[(((training_df.ana==1) & (training_df.pericardial_effusion==1)) & (training_df.proteinuria==1))& (training_df.joint_involvement==0)]
len(commonest_path3_training_df)

1762

In [22]:
len(commonest_path3_training_df[commonest_path3_training_df.label==1])/len(commonest_path3_training_df)*100

94.32463110102157

#### Commonest pathway 4

In [27]:
commonest_path4 = test_df.trajectory.value_counts().index[3]
commonest_path4_net = draw_one_trajectory(test_df, commonest_path4)
commonest_path4_net.show('Example.html')

Example.html


In [24]:
len(test_df[test_df.trajectory == commonest_path4])

459

In [25]:
utils.get_pathway_score(commonest_path4)

0.7671957671957672

In [30]:
# commonest_path4_training_df = training_df[((((((((((((((((((((training_df.ana==1) & (training_df.proteinuria==0)) & (training_df.joint_involvement==0)) & (training_df.pericardial_effusion==0)) & (training_df.anti_dsdna_antibody==1)) & (training_df.delirium==0)) &(training_df.pleural_effusion==0)) & (training_df.fever==0)) &(training_df.psychosis==0)) & (training_df.low_c3==0)) & (training_df.oral_ulcers==0)) & (training_df.lupus_anti_coagulant==0)) & (training_df.low_c4==0)) & (training_df.anti_cardioliphin_antibodies==0)) & (training_df.non_scarring_alopecia==0)) & (training_df.auto_immune_hemolysis==0)) & (training_df.acute_pericarditis==0)) & (training_df.cutaneous_lupus==0))& (training_df.thrombocytopenia==0)) & (training_df.seizure==0)) & (training_df.anti_β2gp1_antibodies==0)]
commonest_path4_training_df = training_df[((((training_df.ana==1) & (training_df.proteinuria==0)) & (training_df.pericardial_effusion==0)) & (training_df.joint_involvement==1)) & (training_df.delirium==1)]
len(commonest_path4_training_df)

1629

In [32]:
len(commonest_path4_training_df[commonest_path4_training_df.label==1])/len(commonest_path4_training_df)*100

93.92265193370166

#### Commonest pathway 5

In [ ]:
commonest_path5 = test_df.trajectory.value_counts().index[4]
commonest_path5_net = draw_one_trajectory(test_df, commonest_path5)
commonest_path5_net.show('Example.html')

In [ ]:
len(test_df[test_df.trajectory == commonest_path5])

In [ ]:
utils.get_pathway_score(commonest_path5)

In [ ]:
# commonest_path5_training_df = training_df[(((((((((((((((((((((training_df.ana==1) & (training_df.proteinuria==0)) & (training_df.joint_involvement==1)) & (training_df.pericardial_effusion==0)) & (training_df.anti_dsdna_antibody==0)) & (training_df.delirium==0)) &(training_df.pleural_effusion==0)) & (training_df.fever==0)) &(training_df.psychosis==0)) & (training_df.anti_smith_antibody==0)) & (training_df.low_c3==0)) & (training_df.oral_ulcers==0)) & (training_df.lupus_anti_coagulant==0)) & (training_df.low_c4==0)) & (training_df.anti_cardioliphin_antibodies==0)) & (training_df.non_scarring_alopecia==0)) & (training_df.auto_immune_hemolysis==0)) & (training_df.acute_pericarditis==0)) & (training_df.cutaneous_lupus==0))& (training_df.thrombocytopenia==0)) & (training_df.seizure==0)) & (training_df.anti_β2gp1_antibodies==0)]
len(commonest_path5_training_df)

In [ ]:
len(commonest_path5_training_df[commonest_path5_training_df.label==0])/len(commonest_path5_training_df)*100